In [ ]:
! pip install torch

In [ ]:
! pip install transformers

     |████████████████████████████████| 3.8 MB 28.4 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 6.5 MB 12.0 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 52.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.listdir('/content/drive/MyDrive/colab/')

['DMSC.csv']

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer,BertModel
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
bertModel = BertModel.from_pretrained("bert-base-chinese",output_hidden_states = True) 

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/colab/DMSC.csv')
data_comments = data['Comment']
data_rate = data['Star'].map({1:-1,2:-1,3:0,4:1,5:1})
def precess_comments(sentence):
    tmp = tokenizer(sentence,return_tensors='pt',truncation=True,padding='max_length',max_length=60)
    return tmp.data['input_ids'],tmp.data['token_type_ids'],tmp.data['attention_mask']
_data_comments = pd.DataFrame(columns=['input_ids','token_type_ids','attention_mask'])
for (index,i) in enumerate(data_comments):
    input_ids,token_type_ids,attention_mask = precess_comments(i)
    _data_comments = _data_comments.append([{'input_ids':input_ids,'token_type_ids':token_type_ids,'attention_mask':attention_mask}])
# 数据集太大了，就弄了一部分

KeyboardInterrupt: ignored

In [ ]:
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [ ]:
_data_rate = data_rate[0:_data_comments.shape]

In [ ]:
import pynvml
pynvml.nvmlInit()
# 这里的0是GPU id
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(meminfo.used)
pynvml.nvmlShutdown()



1849688064


In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.seq=torch.nn.Sequential(
            torch.nn.Linear(768,256),
            torch.nn.ReLU(),
            torch.nn.Linear(256,128),
            torch.nn.ReLU(),
            torch.nn.Linear(128,3),
            torch.nn.Softmax()
        )
    def forward(self,input_ids,toke_type_ids,attention_mask):
        input_ids = input_ids.reshape(-1,60)
        toke_type_ids = toke_type_ids.reshape(-1,60)
        attention_mask = attention_mask.reshape(-1,60)
        out = bertModel(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=toke_type_ids)
        o = self.seq(out.last_hidden_state[:,0])
        return o

class MyDataSet(Dataset):
    def __init__(self,x,y):
        self.train_x = x
        self.train_y = y
    def __len__(self):
        return len(self.train_x)
    def __getitem__(self, item):
        tmp = self.train_y[item]
        if tmp ==0:
            _label_ = [0,1,0]
        elif tmp ==1:
            _label_ = [0,0,1]
        else:
            _label_=[1,0,0]
        return self.train_x.iloc[item,0],self.train_x.iloc[item,1],self.train_x.iloc[item,2],torch.tensor(_label_)
for parameter in bertModel.parameters():
    parameter.requires_grad_(False)
bertModel = bertModel.to(device)


In [ ]:
label = torch.from_numpy(np.array(_data_rate))
myDataset = MyDataSet(_data_comments,_data_rate)
train_size = int(len(myDataset)*0.7)
test_size = int(len(myDataset)*0.2)
valid_size = len(myDataset) - train_size-test_size
train_set,test_set,valid_set = torch.utils.data.random_split(dataset=myDataset,lengths=[train_size,test_size,valid_size])
train_dataLoader = DataLoader(train_set,batch_size=256,shuffle=True)
valid_dataLoader = DataLoader(valid_set,batch_size=256,shuffle=True)


In [ ]:
epochs = 50
criterion = torch.nn.BCELoss()
model = Model()
opt = torch.optim.Adam(params=model.parameters(),lr=0.005)
model = model.to(device)
for epoch in range(epochs):
    l=[]
    acc = []
    correct=0
    for batch_id,(input_id,token_type_id,mask_attention,target) in tqdm(enumerate(train_dataLoader)):
        input_id = input_id.to(device)
        token_type_id = token_type_id.to(device)
        mask_attention = mask_attention.to(device)
        target = target.to(device)
        target = target.type(torch.float)
        o = model(input_id,token_type_id,mask_attention)
        loss = criterion(o,target)
        opt.zero_grad()
        loss.backward()
        opt.step()
        l.append(loss.item())
    for batch_id,(input_id,token_type_id,mask_attention,target) in tqdm(enumerate(valid_dataLoader)):
        input_id = input_id.to(device)
        token_type_id = token_type_id.to(device)
        mask_attention = mask_attention.to(device)
        target = target.to(device)
        target = target.type(torch.float)
        o = model(input_id,token_type_id,mask_attention)
        prediction = torch.argmax(o,dim=1)
        real_class = torch.argmax(target,dim=1)
        correct+=(prediction==real_class).sum().item()
    print(f'epoch ---{epoch}-----loss---{np.mean(l)}---acc:{correct/len(valid_set)}')


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
499it [06:56,  1.20it/s]
72it [01:01,  1.17it/s]


epoch ---0-----loss---0.512513572204567---acc:0.6067083196317001


499it [07:11,  1.16it/s]
72it [01:01,  1.16it/s]


epoch ---1-----loss---0.4937783701266937---acc:0.6134495231831635


499it [07:12,  1.15it/s]
72it [01:01,  1.17it/s]


...

epoch ---3-----loss---0.4834644412827157---acc:0.6170667543571193


499it [07:13,  1.15it/s]
72it [01:01,  1.17it/s]


epoch ---4-----loss---0.48053084036152444---acc:0.6148744930395703


378it [05:29,  1.15it/s]


KeyboardInterrupt: ignored

In [ ]:
tmp = tokenizer("这个电影很一般般",return_tensors='pt',max_length=60,padding='max_length')
o = model(tmp.data['input_ids'].to(device),tmp.data['token_type_ids'].to(device),tmp.data['attention_mask'].to(device))
print(o)
print(['负面','中性','正面'][torch.argmax(o)])

tmp = tokenizer("这个电影很好看",return_tensors='pt',max_length=60,padding='max_length')
o = model(tmp.data['input_ids'].to(device),tmp.data['token_type_ids'].to(device),tmp.data['attention_mask'].to(device))
print(o)
print(['负面','中性','正面'][torch.argmax(o)])

tmp = tokenizer("这个电影很难看",return_tensors='pt',max_length=60,padding='max_length')
o = model(tmp.data['input_ids'].to(device),tmp.data['token_type_ids'].to(device),tmp.data['attention_mask'].to(device))
print(o)
print(['负面','中性','正面'][torch.argmax(o)])

tensor([[0.3047, 0.5432, 0.1521]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
中性
tensor([[0.0067, 0.0201, 0.9732]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
正面
tensor([[0.4895, 0.3770, 0.1334]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
负面


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
correct=0
test_dataLoader = DataLoader(test_set,batch_size=512,shuffle=True)
for batch_id,(input_id,token_type_id,mask_attention,target) in tqdm(enumerate(test_dataLoader)):
    input_id = input_id.to(device)
    token_type_id = token_type_id.to(device)
    mask_attention = mask_attention.to(device)
    target = target.to(device)
    target = target.type(torch.float)
    o = model(input_id,token_type_id,mask_attention)
    prediction = torch.argmax(o,dim=1)
    real_class = torch.argmax(target,dim=1)
    correct+=(prediction==real_class).sum().item()

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
72it [01:56,  1.62s/it]


In [ ]:
print('测试集准确率 {acc}'.format(acc=correct/test_size))

测试集准确率 0.6140480692811532
